# Part 1:Web Scrape from Wikipedia

In [153]:
from bs4 import BeautifulSoup
from lxml import html
import requests
import pandas as pd  
import numpy as np
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!pip install folium
import folium


In [2]:
#retrieving html from link and converting to soup object
site = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
content = site.content
soup = BeautifulSoup(content)

In [3]:
#only storing the indiviual table entries into a new list. Excluding table entries including the word "Not" to remove "Not Assigned" neighborhoods/
lines = [i for table in soup.find_all('table')[:1] for line in table for i in line if "Not" not in str(i)]     

In [4]:
#Extracting data and storing it in dictionary for easy conversion to dataframe
data = {'Postal_Code':[], 'Borough':[],'Neighborhood':[]}
for i in range(len(lines)):
    zone_list = str(lines[i]).split('<td>')
    if len(zone_list) > 1:
        data['Postal_Code'].append(str(lines[i]).split('<td>')[1].rstrip('\n</td>\n'))
        data['Borough'].append(str(lines[i]).split('<td>')[2].rstrip('\n</td>\n'))
        data['Neighborhood'].append(str(lines[i]).split('<td>')[3].rstrip('\n</td></tr>'))

In [156]:
df = pd.DataFrame(data)
df.rename(columns = {'Postal_Code':'Postal Code'}, inplace = True)
df.head()

Postal Code           Borough                                Neighborhood
0         M3A        North York                                   Parkwoods
1         M4A        North York                            Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfron
3         M6A        North York            Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Governmen

In [157]:
df.shape

(103, 3)

# Part 2: Adding latitude and longitude to dataframe from 'Geospatial_Coordinates.csv' file

### Importing csv using IBM functions 

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
#joining both the web scraped dataframe and the latitude and longitude dataframe
final = pd.merge(df,lat_long,on = 'Postal Code')

In [10]:
final.head()

Postal Code           Borough                                Neighborhood  \
0         M3A        North York                                   Parkwoods   
1         M4A        North York                            Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfron   
3         M6A        North York            Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Governmen   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [11]:
final.shape

(103, 5)

# Part 3: Map Visualization of Data

### Let's take a look at only the neighborhoods in Toronto 

In [12]:
toronto = final[final['Borough'] == 'Downtown Toronto']
toronto

Postal Code           Borough  \
2          M5A  Downtown Toronto   
4          M7A  Downtown Toronto   
9          M5B  Downtown Toronto   
15         M5C  Downtown Toronto   
20         M5E  Downtown Toronto   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
30         M5H  Downtown Toronto   
36         M5J  Downtown Toronto   
42         M5K  Downtown Toronto   
48         M5L  Downtown Toronto   
80         M5S  Downtown Toronto   
84         M5T  Downtown Toronto   
87         M5V  Downtown Toronto   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
99         M4Y  Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  
2                            Regent Park, Harbourfron  43.654260 -79.360636  
4          Queen's Park, Ontario Provincial Governmen  43.662301 -79.389494  
9                            Garden District, Ryerson  43.657162 -79.378937  
15                                     St. James Town  43.651494 -79.375418  
20                                        Berczy Park  43.644771 -79.373306  
24                                  Central Bay Stree  43.657952 -79.387383  
25                                           Christie  43.669542 -79.422564  
30                           Richmond, Adelaide, King  43.650571 -79.384568  
36  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752  
42           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576  
48                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
80                       University of Toronto, Harbo  43.662696 -79.400049  
84          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049  
87  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420  
91                                           Rosedale  43.679563 -79.377529  
92                                     Stn A PO Boxes  43.646435 -79.374846  
96                        St. James Town, Cabbagetown  43.667967 -79.367675  
97             First Canadian Place, Underground city  43.648429 -79.382280  
99                               Church and Wellesley  43.665860 -79.383160

### Getting Coordinates

In [16]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Map of Toronto with Neighborhoods superimposed on top

In [17]:
toronto_map =  folium.Map(location=[latitude,longitude], zoom_startss = 10)

for lat, long, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label_string = f'{borough},{neighborhood}'
    label = folium.Popup(label_string, parse_html=True)
    folium.CircleMarker(
        [lat,long],
        popup = label,
        color = 'blue',
        fill=True,
        fill_color = '#3186cc',
        fill_opacity = .7,
        parse_html = False
    
    ).add_to(toronto_map)


In [18]:
toronto_map

# Part 4: FourSquare Venue Data
##### Making API calls, and creating onehot-encoded dataframe for later clustering 

In [19]:
CLIENT_ID = '5CKIKDUK5VHQNELLTKUFNNOHKVISYJ2MZOS2JMYCVDUTH321' # your Foursquare ID
CLIENT_SECRET = 'XUOXALH4INIFZ000T42YVRIUPAPWBGFGOAWV4S2IJMOUM3WY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5CKIKDUK5VHQNELLTKUFNNOHKVISYJ2MZOS2JMYCVDUTH321
CLIENT_SECRET:XUOXALH4INIFZ000T42YVRIUPAPWBGFGOAWV4S2IJMOUM3WY


In [83]:
def get_category(response, i):
    return response['groups'][0]['items'][i]['venue']['categories'][0]['name']

def get_name(response, i):
    return response['groups'][0]['items'][i]['venue']['name']
    
def get_coordinates(response, i):
    return (response['groups'][0]['items'][i]['venue']['location']['lat'],response['groups'][0]['items'][i]['venue']['location']['lng'])
            
            

            


### Getting venue info from API

In [84]:

radius = 500
LIMIT = 100
venue_list = []

#loops through each neighborhood
for i in range(toronto.shape[0]):
    neighborhood = toronto.iloc[i,-3]
    neighborhood_latitude = toronto.iloc[i,-2]
    neighborhood_longitude = toronto.iloc[i,-1]
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
    
    #response with venues for each neighborhood
    response = requests.get(url).json()['response']
    
    #loops through each venue in response, and saves venue info to dictionary
    for venue in range(len(response['groups'][0]['items'])):
        venue_dict = {}
        venue_dict['neighborhood'] = neighborhood
        venue_dict['category'] = get_category(response, venue)
        venue_dict['name'] = get_name(response, venue)
        venue_dict['lat'] = get_coordinates(response, venue)[0]
        venue_dict['long'] = get_coordinates(response, venue)[1]
        venue_list.append(venue_dict)



      

In [25]:
#venue dictionary is turned into dataframe
venues_df = pd.DataFrame(venue_list)
venues_df.head(10)

category        lat       long  \
0                Bakery  43.653447 -79.362017   
1           Coffee Shop  43.653559 -79.361809   
2        Breakfast Spot  43.653947 -79.361149   
3   Distribution Center  43.653249 -79.358008   
4                   Spa  43.654735 -79.359874   
5            Restaurant  43.656369 -79.356980   
6                  Park  43.655618 -79.356211   
7  Gym / Fitness Center  43.653313 -79.359725   
8         Historic Site  43.650244 -79.359323   
9        Breakfast Spot  43.655675 -79.364503   

                               name              neighborhood  
0                  Roselle Desserts  Regent Park, Harbourfron  
1                     Tandem Coffee  Regent Park, Harbourfron  
2                Morning Glory Cafe  Regent Park, Harbourfron  
3            Cooper Koo Family YMCA  Regent Park, Harbourfron  
4               Body Blitz Spa East  Regent Park, Harbourfron  
5                    Impact Kitchen  Regent Park, Harbourfron  
6                   Corktown Common  Regent Park, Harbourfron  
7                The Extension Room  Regent Park, Harbourfron  
8  The Distillery Historic District  Regent Park, Harbourfron  
9            Figs Breakfast & Lunch  Regent Park, Harbourfron

### One-hot encoding

In [90]:
#one-hot encoding
toronto_onehot = pd.get_dummies(venues_df, columns = ['category'] ).drop(columns = ['lat','long','name'])
toronto_cat = toronto_onehot.groupby('neighborhood').mean()
toronto_cat

category_Accessories Store  \
neighborhood                                                                     
Berczy Park                                                           0.000000   
CN Tower, King and Spadina, Railway Lands, Harb...                    0.000000   
Central Bay Stree                                                     0.000000   
Christie                                                              0.000000   
Church and Wellesley                                                  0.000000   
Commerce Court, Victoria Hotel                                        0.000000   
First Canadian Place, Underground city                                0.000000   
Garden District, Ryerson                                              0.000000   
Harbourfront East, Union Station, Toronto Islands                     0.000000   
Kensington Market, Chinatown, Grange Park                             0.000000   
Queen's Park, Ontario Provincial Governmen                            0.000000   
Regent Park, Harbourfron                                              0.000000   
Richmond, Adelaide, King                                              0.010638   
Rosedale                                                              0.000000   
St. James Town                                                        0.000000   
St. James Town, Cabbagetown                                           0.000000   
Stn A PO Boxes                                                        0.000000   
Toronto Dominion Centre, Design Exchange                              0.000000   
University of Toronto, Harbo                                          0.000000   

                                                    category_Afghan Restaurant  \
neighborhood                                                                     
Berczy Park                                                           0.000000   
CN Tower, King and Spadina, Railway Lands, Harb...                    0.000000   
Central Bay Stree                                                     0.000000   
Christie                                                              0.000000   
Church and Wellesley                                                  0.013699   
Commerce Court, Victoria Hotel                                        0.000000   
First Canadian Place, Underground city                                0.000000   
Garden District, Ryerson                                              0.000000   
Harbourfront East, Union Station, Toronto Islands                     0.000000   
Kensington Market, Chinatown, Grange Park                             0.000000   
Queen's Park, Ontario Provincial Governmen                            0.000000   
Regent Park, Harbourfron                                              0.000000   
Richmond, Adelaide, King                                              0.000000   
Rosedale                                                              0.000000   
St. James Town                                                        0.000000   
St. James Town, Cabbagetown                                           0.000000   
Stn A PO Boxes                                                        0.000000   
Toronto Dominion Centre, Design Exchange                              0.000000   
University of Toronto, Harbo                                          0.000000   

                                                    category_Airport  \
neighborhood                                                           
Berczy Park                                                   0.0000   
CN Tower, King and Spadina, Railway Lands, Harb...            0.0625   
Central Bay Stree                                             0.0000   
Christie                                                      0.0000   
Church and Wellesley                                          0.0000   
Commerce Court, Victoria Hotel                                0.0000   
First Canadian Place, Undergro

### Getting top 10 venues for each neighborhood

In [159]:
#getting top 10 venues for each neighborhood
top_10 = pd.DataFrame()
for neigh in toronto_cat.index:
    top_10 = top_10.append(pd.DataFrame([list(toronto_cat.loc[neigh,:].nlargest(10).index)]))
top_10['neighborhood'] = toronto_cat.index
top_10.rename(columns = {
                        0:'1st Most Popular Venue',
                        1:'2nd Most Popular Venue',
                        2:'3rd Most Popular Venue',
                        3:'4th Most Popular Venue',
                        4:'5th Most Popular Venue',
                        5:'6th Most Popular Venue',
                        6:'7th Most Popular Venue',
                        7:'8th Most Popular Venue',
                        8:'9th Most Popular Venue',
                        9:'10th Most Popular Venue'},
                        inplace = True)

#reorganizing order or column names 
top_10 = top_10[['neighborhood','1st Most Popular Venue', '2nd Most Popular Venue',
       '3rd Most Popular Venue', '4th Most Popular Venue',
       '5th Most Popular Venue', '6th Most Popular Venue',
       '7th Most Popular Venue', '8th Most Popular Venue',
       '9th Most Popular Venue', '10th Most Popular Venue']]
top_10.reset_index(drop = True, inplace = True)
top_10.head()

neighborhood  \
0                                        Berczy Park   
1  CN Tower, King and Spadina, Railway Lands, Har...   
2                                  Central Bay Stree   
3                                           Christie   
4                               Church and Wellesley   

     1st Most Popular Venue        2nd Most Popular Venue  \
0      category_Coffee Shop         category_Cocktail Bar   
1  category_Airport Service       category_Airport Lounge   
2      category_Coffee Shop                 category_Café   
3    category_Grocery Store                 category_Café   
4      category_Coffee Shop  category_Japanese Restaurant   

        3rd Most Popular Venue       4th Most Popular Venue  \
0              category_Bakery            category_Beer Bar   
1             category_Airport  category_Airport Food Court   
2  category_Italian Restaurant        category_Dessert Shop   
3                category_Park  category_Athletics & Sports   
4    category_Sushi Restaurant          category_Restaurant   

    5th Most Popular Venue     6th Most Popular Venue  \
0            category_Café       category_Cheese Shop   
1    category_Airport Gate  category_Airport Terminal   
2  category_Sandwich Place               category_Bar   
3      category_Baby Store       category_Candy Store   
4       category_Gastropub           category_Gay Bar   

     7th Most Popular Venue             8th Most Popular Venue  \
0       category_Restaurant        category_Seafood Restaurant   
1              category_Bar             category_Boat or Ferry   
2  category_Bubble Tea Shop              category_Burger Joint   
3      category_Coffee Shop                     category_Diner   
4            category_Hotel  category_Mediterranean Restaurant   

        9th Most Popular Venue       10th Most Popular Venue  
0         category_Art Gallery            category_BBQ Joint  
1            category_Boutique          category_Coffee Shop  
2      category_Ice Cream Shop  category_Japanese Restaurant  
3  category_Italian Restaurant            category_Nightclub  
4         category_Men's Store                  category_Pub

# Part 5: Clustering 

#### DataFrame for Clustering

In [139]:
toronto_clustering = toronto_cat.reset_index().drop('neighborhood', axis = 1)

#### Preparing model

In [147]:
clusters = 4
kmod = KMeans(n_clusters = clusters, random_state = 0).fit(toronto_clustering)
kmod.labels_

array([0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
      dtype=int32)

#### Merging DataFrames to visualize information, and also adding cluster labels

In [148]:
toronto_merged = pd.merge(toronto,top_5,left_on='Neighborhood', right_on = 'neighborhood').sort_values('Neighborhood')
toronto_merged['Cluster'] = kmod.labels_
toronto_merged = toronto_merged[['Postal Code', 'Borough', 'Neighborhood','Cluster', 'Latitude', 'Longitude',
       'neighborhood', '1st Most Popular Venue', '2nd Most Popular Venue',
       '3rd Most Popular Venue', '4th Most Popular Venue',
       '5th Most Popular Venue', '6th Most Popular Venue',
       '7th Most Popular Venue', '8th Most Popular Venue',
       '9th Most Popular Venue', '10th Most Popular Venue']]

In [149]:
toronto_merged.head()

Postal Code           Borough  \
4          M5E  Downtown Toronto   
13         M5V  Downtown Toronto   
5          M5G  Downtown Toronto   
6          M6G  Downtown Toronto   
18         M4Y  Downtown Toronto   

                                         Neighborhood  Cluster   Latitude  \
4                                         Berczy Park        0  43.644771   
13  CN Tower, King and Spadina, Railway Lands, Har...        3  43.628947   
5                                   Central Bay Stree        0  43.657952   
6                                            Christie        2  43.669542   
18                               Church and Wellesley        0  43.665860   

    Longitude                                       neighborhood  \
4  -79.373306                                        Berczy Park   
13 -79.394420  CN Tower, King and Spadina, Railway Lands, Har...   
5  -79.387383                                  Central Bay Stree   
6  -79.422564                                           Christie   
18 -79.383160                               Church and Wellesley   

      1st Most Popular Venue        2nd Most Popular Venue  \
4       category_Coffee Shop         category_Cocktail Bar   
13  category_Airport Service       category_Airport Lounge   
5       category_Coffee Shop                 category_Café   
6     category_Grocery Store                 category_Café   
18      category_Coffee Shop  category_Japanese Restaurant   

         3rd Most Popular Venue       4th Most Popular Venue  \
4               category_Bakery            category_Beer Bar   
13             category_Airport  category_Airport Food Court   
5   category_Italian Restaurant        category_Dessert Shop   
6                 category_Park  category_Athletics & Sports   
18    category_Sushi Restaurant          category_Restaurant   

     5th Most Popular Venue     6th Most Popular Venue  \
4             category_Café       category_Cheese Shop   
13    category_Airport Gate  category_Airport Terminal   
5   category_Sandwich Place               category_Bar   
6       category_Baby Store       category_Candy Store   
18       category_Gastropub           category_Gay Bar   

      7th Most Popular Venue             8th Most Popular Venue  \
4        category_Restaurant        category_Seafood Restaurant   
13              category_Bar             category_Boat or Ferry   
5   category_Bubble Tea Shop              category_Burger Joint   
6       category_Coffee Shop                     category_Diner   
18            category_Hotel  category_Mediterranean Restaurant   

         9th Most Popular Venue       10th Most Popular Venue  
4          category_Art Gallery            category_BBQ Joint  
13            category_Boutique          category_Coffee Shop  
5       category_Ice Cream Shop  category_Japanese Restaurant  
6   category_Italian Restaurant            category_Nightclub  
18         category_Men's Store                  category_Pub

#### Create Cluster Map

In [155]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters